In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dask
dask.config.set({'temporary_directory': 'Z:/tmp'})
import dask.dataframe as dd

from datetime import timedelta
import tqdm
import pickle
import math

pd.set_option('display.max_columns', None)

In [ ]:
event_headers = pd.read_csv('../data/raw/acquia_samples/scripts-headers/event-headers.txt', delimiter='\t')
event_columns = event_headers.columns
event_columns = [col for col in event_columns] + [f'extra_{i}' for i in range(1, 7)]

In [ ]:
event_columns[47]

In [ ]:
event = dd.read_csv('Z:\\event_fixed.tsv', sep='\t', names=event_columns,
                   dtype={'account_id': str, 'content_section.1': str, 'download_link': str, 'role': str,
                         'get-legal-help': str, 'keywords': str, 'Outgoing_link': str, 'Language': str,
                         'legal_position': str}).set_index('event_id')

In [ ]:
drop_columns = ([f'extra_{i}' for i in range(1, 7)]+
                ['empty']+
                [f'empty{i}' for i in range(2, 8)]+
                [f'empty{i}' for i in range(10, 12)]+
                [f'empty{i}' for i in range(15, 28)]+
                [f'Unnamed: {i}' for i in range(60, 103)]+
                [f'Unnamed: {i}' for i in range(104, 108)])

event = event.drop(drop_columns, axis=1)

In [ ]:
people = pd.read_csv("Z:\\people_business_desktop.tsv", sep='\t')
people

In [ ]:
people_ids = people['person_id']

In [ ]:
people_ids

In [ ]:
event_people_ids = event['person_id'].compute()

In [ ]:
event = event[event_people_ids.isin(people_ids)]

In [ ]:
len(event)

In [ ]:
event.head()

In [ ]:
event.to_csv("Z:\\event_selected\\*.tsv", sep='\t')

In [ ]:
event = dd.read_csv('Z:\\event_selected\\*.tsv', sep='\t',
                    dtype={'account_id': str, 'content_section.1': str, 'download_link': str, 'role': str,
                         'get-legal-help': str, 'keywords': str, 'Outgoing_link': str, 'Language': str,
                         'legal_position': str}).set_index('event_id')

In [ ]:
touch_busydesk = pd.read_csv("Z:\\people_business_desktop.tsv", sep='\t')
touch_busydesk.set_index('person_id', inplace=True)

In [ ]:
touch_busydesk

In [ ]:
mostly_desktop = (touch_busydesk['is_desktop'] > .5)
desktop_ppl = touch_busydesk[mostly_desktop].index.values

In [ ]:
len(desktop_ppl)

In [ ]:
desktop_events = event[event['person_id'].isin(desktop_ppl)].compute()
desktop_events.to_csv("Z:\\event_desktop_ppl2.tsv", sep='\t')

In [ ]:
len(desktop_events)

In [ ]:
mobile_events = event[~event['person_id'].isin(desktop_ppl)].compute()
mobile_events.to_csv("Z:\\event_mobile_ppl2.tsv", sep='\t')

In [ ]:
len(mobile_events)

In [ ]:
is_professional = (touch_busydesk['business_hours'] > .5) & (touch_busydesk['is_desktop'] > .5)
professionals = touch_busydesk[is_professional].index.values
professional_events = event[event['person_id'].isin(professionals)].compute()
professional_events.to_csv("Z:\\event_professionals2.tsv", sep='\t')

In [ ]:
len(professional_events)

In [ ]:
nonprofessional_events = event[~event['person_id'].isin(professionals)].compute()
nonprofessional_events.to_csv("Z:\\event_nonprofessionals2.tsv", sep='\t')

In [ ]:
len(nonprofessional_events)